In [325]:
import random
import numpy as np
from collections import deque

In [326]:
def getlambda0(t):
    # t está en minutos, convertir a horas:
    t = t/60
    t = t%24 ##########ojo esto##########
    # Intervalo de 0 a 5 horas, lineal de 5 a 7
    if 0 <= t < 5:
        return 5 + (7 - 5) * (t - 0) / (5 - 0)
    # Intervalo de 5 a 8 horas, lineal de 7 a 6
    elif 5 <= t < 8:
        return 7 + (6 - 7) * (t - 5) / (8 - 5)
    # Intervalo de 8 a 15 horas, lineal de 6 a 9
    elif 8 <= t < 15:
        return 6 + (9 - 6) * (t - 8) / (15 - 8)
    # Intervalo de 15 a 17 horas, lineal de 9 a 6
    elif 15 <= t < 17:
        return 9 + (6 - 9) * (t - 15) / (17 - 15)
    # Intervalo de 17 a 24 horas, lineal de 6 a 5
    elif 14 <= t <= 24:
        return 6 + (5 - 6) * (t - 17) / (24 - 17)
    # Si el tiempo está fuera del rango de 0 a 24 horas
    else:
        return None  

In [327]:
# T = 24*60 # 24h en minutos
T = 30*24*60 #un mes en minutos

nPistas = 3
nCoches = 20
nPuertas = 50

# lambda0 = getlambda0(t)
mu11 = 10.007615178514001
sigma11 = 3.0377002745229005
a12 = 10
b12 = 15
lambda2 = 2
lambda3 = 0.1

Comprobamos que funcionen las distribuciones

In [328]:
# Funciones auxiliares generacion de tiempos
def t_llegada(t):
    return random.expovariate(getlambda0(t))*60 #nº muy pequeños, tiene sentido?

def t_aterrizaje():
    # return np.random.normal(mu11, sigma11)
    # return random.normalvariate(mu11, sigma11) 
    while True:  # Bucle infinito hasta obtener un número válido
        resultado = np.random.normal(mu11, sigma11)
        if resultado > 0:  # Comprobar si el resultado es mayor que cero
            return resultado
        
def t_traslado(): #N1- N2
    return np.random.exponential(1/lambda2)

def t_desembarques():
    return np.random.exponential(1/lambda3)

def t_salidas():
    return np.random.uniform(a12, b12)

In [329]:
print("Lambda al inicio de la simulación es " + str(getlambda0(0)) + 
      ", al final es " + str(getlambda0(T/2)) + ' y a la mitad es ' + str(getlambda0(T/2)))

print('Tiempo de llegada: ' + str(t_llegada(T)))
print('Tiempo de llegada: ' +   str(t_llegada(T/2)))
print('Tiempo de llegada: ' +     str(t_llegada(T)))

Lambda al inicio de la simulación es 5.0, al final es 5.0 y a la mitad es 5.0
Tiempo de llegada: 8.658434284239162
Tiempo de llegada: 1.2696979710788745
Tiempo de llegada: 1.4974746213867594


Variables

In [330]:
t = 0                   # Variable reloj. tiempo asíncrono.
n1 = n2 = n3 = 0        # Variables de estado nº clientes
NLL1 = NLL2 = NLL3 = 0  # Variables contador nº llegadas
NS1 = NS2 = NS3 = 0     # Variables contador nº salidas

# Lista de tiempos de eventos
LISTA = {}
LISTA['tLL1'] = LISTA['tS1'] = LISTA['tS2'] = LISTA['tS3'] = LISTA['tSS'] = -1

# Arrays for arrival and departure times
LL1 = []
LL2 = []
LL3 = []
S1 = []
S2 = []
S3 = []

n_med_n1 = n_med_n2 = n_med_n3 = 0 #nº medio de pistas ocupadas
nmed_pistasOcupadas = 0

n_cola_aterrizaje = 0 # Contador de aviones esperando para aterrizar
pistasOcupadas = 0

colaN2 = 0

#variables de estado
todas_las_pistas_ocupadas = True
coches_disponibles = True
puertas_embarque_disponibles = True

In [331]:
def rutina_aterrizaje(t_suc):
    
    global t, n1, n2, n3, NLL1, LL1, LISTA
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, colaN2, pistasOcupadas,n_cola_aterrizaje
    
    print(f'----Llega un vuelo en el minuto {t_suc}----')
    # Actualizo variables contadores
    n1 += 1                     # aviones en n1
    NLL1 += 1                   # numero de llegadas al n1
    LISTA['tLL1'] = t_suc # Guardo el tiempo del evento en la lista de tiempos LL1
    t = t_suc                   # Actualizo reloj asíncrono
    
    # para luego calcular numero medio de clientes
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    nmed_pistasOcupadas += pistasOcupadas * (t_suc - t)
    
    
    # Si tengo pista libre, aterrizo
    if (pistasOcupadas < nPistas):
        Z = t_aterrizaje() #random.normalvariate(mu11, sigma11) #genero tiempo aterrizaje
        LISTA['tS1'] = t + Z #tiempo de salida del N1, que se corresponde con el final de aterrizaje
        pistasOcupadas+=1
        print(f'Avion finaliza aterrizaje en el instante {t+Z}')
        #entro en cola para el traslado
        colaN2+=1
    
    else:
        #si no tengo pista, no aterrizo, me pongo en cola aterrizaje
        n_cola_aterrizaje+=1

    #=========================================================================#

    # Genero tiempo siguente llegada -> esto tiene sentido hacerlo aqui?
    Y = t_llegada(t)
    # Compruebo si el siguiente avión llega antes de T
    if (t + Y < T):
        LISTA['tLL1'] = t + Y #añado otro tiempo de llegada
        print(f'El siguiente avion llegará en el instante {t+Y}')
    
    else:
        print(f'Horizonte alcanzado {t + Y} , {T}, no llegan más aviones')

In [332]:
def rutina_traslado(t_suc): 
    # un coche recoge al avión y lo desaloja de la pista (salida de N1)
    global t, n1, n2, n3, NLL1, LL1, LISTA
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, colaN2, pistasOcupadas,n_cola_aterrizaje, coches_disponibles, nPista
    
    t=t_suc #Actualizamos tº de simulación
    pistasOcupadas-=1 #DESPEJO LA PISTA EN LA QUE ESTABA EL AVION
    n1 -=1 #Decrementamos el nº de aviones del nodo 1 porque el avión sale del nodo 1.
    n2 +=1 #Incrementamos el nº de aviones en nodo 2
    
    LISTA['tS1'] = t_suc #Registramos el instante en el que NS1 sale del nodo 1.
    
    # para luego calcular numero medio de clientes
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    
    #si hay coches disponibles -> traslado
    if coches_disponibles > 0:
        coches_disponibles -=1 #tambien lo puedo hacer con num vehiculos
        Z = t_traslado()
        print(f'----Comienza el traslado que durará {Z}----')
        print(f'----El avión llega a la zona de desembarque en el instante {t + Z}----')
        print(f'----Ahora hay {coches_disponibles} coches disponibles')
        LISTA['tS2'] = t + Z # tiempo de salida del N2, que se corresponde con el final del traslado
        
    else: #si no hay coche disponibles 
        print('----no hay vehículos disponibles---')
        colaN2+=1 #lo pongo a la cola de ser trasladado
    
    if colaN2 > 0:
        #que hago aqui
        return

In [333]:
def rutina_desembarque(t_suc): 
    # avion en aeropuerto: desembarcan pasajeros, embarcan los nuevos y traslado a pistas
    global t, n1, n2, n3, NLL1, LL1, LISTA
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, colaN2, pistasOcupadas,n_cola_aterrizaje, coches_disponibles, nPista
    
    t=t_suc #Actualizamos tº de simulación
    n2 -=1 #Decrementamos el nº de aviones del nodo 2 porque el avión sale del nodo 2.
    n3 +=1 #Incrementamos el nº de aviones en nodo 3
    
    LISTA['tS2'] = t_suc #esto tiene sentido hacerlo así?
    
    # para luego calcular numero medio de clientes
    n_med_n1 += n1 * (t_suc - t) #tiempo trascurrido hasta el suceso * nº aviones
    n_med_n2 += n2 * (t_suc - t)
    n_med_n3 += n3 * (t_suc - t)
    
    #si hay puertas de embarque disponibles y pistas disponibles (esto no sé muy bien como representarlo)
    if puertas_embarque_disponibles > 0:
        puertas_embarque_disponibles -=1 #tambien lo puedo hacer con num vehiculos
        Z = t_desembarques()
        print(f'----Comienza el cambio de pasajeros y traslado a pistas que durará {Z}----')
        
        if pistasOcupadas < nPistas:
            print(f'----El avión llega de nuevo a las pistas en el instante {t + Z}----')
            LISTA['tS2'] = t + Z # tiempo de salida del N3, que se corresponde con la vuelta a la pista
    
        print(f'----Ahora hay {puertas_embarque_disponibles} coches disponibles')
        
    else: #si no hay puertas disponibles 
        print('----no hay puertas de embarque disponibles---')


In [334]:
def simul_main():
    global t, n1, n2, n3, NLL1, LL1, LISTA
    global n_med_n1, n_med_n2, n_med_n3, nmed_pistasOcupadas, colaN2, pistasOcupadas,n_cola_aterrizaje, nPistas, coches_disponibles
    
    t = 0                   # Variable reloj. tiempo asíncrono.
    n1 = n2 = n3 = 0        # Variables de estado nº clientes
    NLL1 = NLL2 = NLL3 = 0  # Variables contador nº llegadas
    NS1 = NS2 = NS3 = 0     # Variables contador nº salidas

    nPistas = 3
    coches_disponibles = 20
    # Lista de tiempos de eventos
    # LISTA = {}
    # LISTA['tLL1'] = LISTA['tS1'] = LISTA['tS2'] = LISTA['tS3'] = LISTA['tSS'] = -1

    # # Arrays for arrival and departure times
    # LL1 = []
    # LL2 = []
    # LL3 = []
    # S1 = []
    # S2 = []
    # S3 = []
    
    LISTA = {
    'tLL1': 10E9,   # Inicializa 'tLL1' como una lista vacía
    'tS1':  10E9,
    'tS2':  10E9,
    'tS3':  10E9,
    'tSS':  10E9
}


    n_med_n1 = n_med_n2 = n_med_n3 = 0 #nº medio de pistas ocupadas
    nmed_pistasOcupadas = 0

    n_cola_aterrizaje = 0 # Contador de aviones esperando para aterrizar
    n_cola_traslado = 0
    pistasOcupadas = 0

    colaN2 = 0

    #variables de estado
    todas_las_pistas_ocupadas = True
    puertas_embarque_disponibles = True

    X = t_llegada(0)
    print(LISTA)
    rutina_aterrizaje(X)
    while (t <= T and (LISTA['tLL1'] or LISTA['tS1'] or LISTA['tS2'] or LISTA['tS3'] or LISTA['tSS'])):
        print(LISTA)
        if(min(LISTA, key=LISTA.get)=='tLL1'): # Si el próximo evento es un aterrizaje (LLN1)
            print('----Ahora toca una llegada----')
            t_suc=LISTA['tLL1']
            LISTA['tLL1']= 10E9
            rutina_aterrizaje(t_suc)
            print(LISTA)
        elif(min(LISTA, key=LISTA.get)=='tS1'): # Si el próximo evento es un traslado (SN1)
            print('----Ahora toca un traslado----')
            t_suc=LISTA['tS1']
            LISTA['tS1']= 10E9
            rutina_traslado(t_suc)
            print(LISTA)
            
        elif(min(LISTA, key=LISTA.get)=='tS2'): # Si el próximo evento es un deembarque (SN2)
            print('----Ahora toca un cambio de pasajeros----')
            t_suc=LISTA['tS2']
            # LISTA['tS2']= 10E9
            # rutina_desembarque(t_suc)
            # print(LISTA)
    

In [335]:

simul_main()

{'tLL1': 10000000000.0, 'tS1': 10000000000.0, 'tS2': 10000000000.0, 'tS3': 10000000000.0, 'tSS': 10000000000.0}
----Llega un vuelo en el minuto 28.642860705613817----
Avion finaliza aterrizaje en el instante 39.37880576745945
El siguiente avion llegará en el instante 36.94316406544463
{'tLL1': 36.94316406544463, 'tS1': 39.37880576745945, 'tS2': 10000000000.0, 'tS3': 10000000000.0, 'tSS': 10000000000.0}
----Ahora toca una llegada----
----Llega un vuelo en el minuto 36.94316406544463----
Avion finaliza aterrizaje en el instante 48.69545746487178
El siguiente avion llegará en el instante 37.44486883097604
{'tLL1': 37.44486883097604, 'tS1': 48.69545746487178, 'tS2': 10000000000.0, 'tS3': 10000000000.0, 'tSS': 10000000000.0}
{'tLL1': 37.44486883097604, 'tS1': 48.69545746487178, 'tS2': 10000000000.0, 'tS3': 10000000000.0, 'tSS': 10000000000.0}
----Ahora toca una llegada----
----Llega un vuelo en el minuto 37.44486883097604----
Avion finaliza aterrizaje en el instante 48.10834387104263
El sig

KeyboardInterrupt: 